# Experiment Utilities

This notebook contains utility functions for experiments, such as data gathering, analysis, and visualisation functions.

## Data Gathering Functions

Contains various functions used to gather experiments data.

In [ ]:
def do_experiment_per_shift(model, method, x_valid, y_valid, 
                            x_test, y_test, shift_type, shift_type_params=None):
    """
    Calculate the test statistics, p-value, and detection accuracy for a given method
    and shift on all combinations of number of test samples, shift intensities, 
    and proportion of test data that is affected by shift.

    :param model: the model can be a dimensionality reductor, CBM, CME, or blackbox classifier
    :param method: the shift detection method which can be:
        - 'BBSDs': softmax label classifier (BBSD)
        - 'BBSDh': argmax/ hard prediction label classifier 
        - 'BBSDs_concepts': softmax on the concept layer 
        - 'BBSDh_concepts': argmax/ hard prediction label classifier
        - 'PCA': used to reduce the dimension of x_valid and x_test
        - 'SRP': used to reduce the dimension of x_valid and x_test
        - 'UAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'TAE': Autoencoder based method to reduce the dimension of x_valid and x_test
        - 'NoRed': original dimensions of x_valid and x_test, no dimensionality reduction applied
    :param x_valid: validation data, which we hypothetically treat as the dataset that we have.
    :param x_test: test data, which we hypothetically treat as unseen real-world data, where shift might occur
    :param shift_type: a different type of shift which can be:
        - 'gaussian'
        - 'ko'
        - 'img': random image shift incorporating combination of below shift
        - 'width_shift'
        - 'height_shift'
        - 'rotation'
        - 'shear'
        - 'zoom'
        - 'flip'
        - If wishing to do combination of shift, give an array comprising the above parameters.
    :param shift_type_params: If needed, provide shift parameters (e.g., the original image size, see apply_shift_* functions)

    :return: a dictionary containing p-value and detection accuracy for all combination of shift intensities,
        shift proportion, and number of test samples:
        {
            "shift_intensities": {
                "shift_proportion": {
                    "test_samples: : {
                        "test_statistics": [],
                        "p_vals": [],
                        "detection_result: []
                    }
                }
            }
        }
    """

    # Possible value of intensities, data proportion affected, test set samples
    shift_intensities = ["small", "medium", "large"]
    shift_props = [0.1, 0.5, 1.0]
    test_set_samples = [10, 20, 50, 100, 200, 500, 1000, 10000]
    n_exp = 5 # number of experiments for each configuration (for reliability)

    ## Initialise dictionary used to store result
    dict_result = initialise_result_dictionary(shift_intensities, shift_props, test_set_samples)

    ############################################################################
    ## BBSDs
    ## Process differently depending on methods 
    if method == "BBSDs":
        # Consider all combinations of shift intensities, shift proportion, test samples
        for shift_intensity in shift_intensities:
            for shift_prop in shift_props:
                for test_set_sample in test_set_samples:
                    # Get test set
                    ## TODO

                    # Call apply shift method on the test set
                    ## TODO

                    # Perform detection:
                    # 1. Get reduced representation
                    # 2. Perform statistical test

                    
                    # 3. Store result
        

In [ ]:
def apply_shift(x_test, y_test, shift_type, shift_type_params, shift_intensity, shift_prop):
    """
    Apply a type of shift to x_test and y_test.

    :param x_valid: validation data, which we hypothetically treat as the dataset that we have.
    :param x_test: test data, which we hypothetically treat as unseen real-world data, where shift might occur
    :param shift_type: a different type of shift which can be:
        - 'gaussian'
        - 'ko'
        - 'img': random image shift incorporating combination of below shift
        - 'width_shift'
        - 'height_shift'
        - 'rotation'
        - 'shear'
        - 'zoom'
        - 'flip'
        - If wishing to do combination of shift, give an array comprising the above parameters.
    :param shift_type_params: If needed, provide shift parameters (e.g., the original image size, see apply_shift_* functions)
    :param shift_intensity: "small", "medium", or "large"

    :return: (x_test_shifted, y_test_shifted)
    """

    # Prevent bugs, just copy the whole thing
    x_test_shifted = deepcopy(x_test)
    y_test = deepcopy(y_test)

In [ ]:
def single_experiment(model, method, x_valid, y_valid, x_test, y_test):
    """
    Used to perform single experiment for a given data. Fast experiment check.
    """

## Visualisation Functions

Contain functions used to generate plots and tables.

## Helper Functions
Contains sub-functions used to aid the primary functions described in the sections above.

In [5]:
def initialise_result_dictionary(shift_intensities, shift_props, test_set_samples):
    """
    Initialise dictionary used to store result of the experiments.
    """
    
    dict_result = dict()

    ## Generate empty dictionary to store
    for shift_intensity in shift_intensities:
        dict_result[shift_intensity] = dict()
        for shift_prop in shift_props:
            dict_result[shift_intensity][shift_prop] = dict()
            for test_set_sample in test_set_samples:
                dict_result[shift_intensity][shift_prop][test_set_sample] = {
                    "test_statistics": [],
                    "p_vals": [],
                    "detection_result": []
                }
    
    return dict_result

In [9]:
def get_data_subset(x, y, delta):
    """
    Get random proportion (subset) of data x and y.
    """